In [1]:
import torch
import torch.nn as nn
import numpy as np
from torchviz import make_dot
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import os

os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin/'

# download https://graphviz.org/download/ untuk membentuk visual

xl = pd.ExcelFile("Data Ta Idol musim 1-12 fixed with label.xlsx")
xl.sheet_names

['Sheet1']

In [2]:
df = xl.parse("Sheet1")
df.head()

,No,Nama Peserta,Lagu & Penyanyi Asli,Hasil,Genre,Babak,Musim,Usia,Gender,Asal Daerah
0,1,Helena Andrian,"Yang Ku 'Tunggu"" - Anggun",1,1,8,1,20,0,1
1,2,Raymond Latuheru,"""Firasat"" - Marcell Siahaan",0,1,8,1,22,1,1
2,3,Tiara Degrasia,"""Kissing A Fool"" - George Michael",0,2,8,1,20,0,3
3,4,Aldy Erlangga,"""One Last Cry"" - Brian McKnight",0,3,8,1,22,1,1
4,5,Imelda Ragillia,"""Cinta Kita"" - Reza Artamevia",0,1,8,1,22,0,1


In [3]:
# Normalisasi Umur dan babak
scaler = MinMaxScaler(feature_range=(0, 1))
df['Usia_Normalized'] = scaler.fit_transform(df[['Usia']])
df['Babak_Normalized'] = scaler.fit_transform(df[['Babak']])
df.head()

,No,Nama Peserta,Lagu & Penyanyi Asli,Hasil,Genre,Babak,Musim,Usia,Gender,Asal Daerah,Usia_Normalized,Babak_Normalized
0,1,Helena Andrian,"Yang Ku 'Tunggu"" - Anggun",1,1,8,1,20,0,1,0.285714,0.294118
1,2,Raymond Latuheru,"""Firasat"" - Marcell Siahaan",0,1,8,1,22,1,1,0.428571,0.294118
2,3,Tiara Degrasia,"""Kissing A Fool"" - George Michael",0,2,8,1,20,0,3,0.285714,0.294118
3,4,Aldy Erlangga,"""One Last Cry"" - Brian McKnight",0,3,8,1,22,1,1,0.428571,0.294118
4,5,Imelda Ragillia,"""Cinta Kita"" - Reza Artamevia",0,1,8,1,22,0,1,0.428571,0.294118


In [4]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

## Pisahkan fitur dan target
X = df[['Genre','Babak','Gender','Asal Daerah','Usia_Normalized','Babak_Normalized']]
y = df['Hasil']


# Encode target variable as one-hot vectors for binary classification (assuming Hasil has values 0 or 1)
y = tf.keras.utils.to_categorical(y, num_classes=2)

# Bagi data menjadi training dan testing set 70 % untuk training dan 30% untuk testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

class NeuralNetwork(tf.keras.Model):
  def __init__(self, input_size, output_size):
    super(NeuralNetwork, self).__init__()
    self.dense = tf.keras.layers.Dense(units=2, activation='sigmoid', input_shape=(input_size,))

  def call(self, inputs):
    return self.dense(inputs)
      
class NeuralNetwork2(tf.keras.Model):
    def __init__(self, input_size, hidden_units, output_size):
        super(NeuralNetwork2, self).__init__()
        self.hidden_layer = tf.keras.layers.Dense(units=hidden_units, activation='relu', input_shape=(input_size,), name='layer_input_z')
        self.output_layer = tf.keras.layers.Dense(units=output_size, activation='sigmoid')

    def call(self, inputs):
        x = self.hidden_layer(inputs)
        return self.output_layer(x)

# Create the model instance
model = NeuralNetwork(input_size=6, output_size=2)
# model = NeuralNetwork2(input_size=6, hidden_units=8, output_size=2)

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_accuracy', patience=5)
# Train the model (assuming your training data is prepared in X_train and y_train)
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Evaluate the model (assuming your testing data is prepared in X_test and y_test)
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy * 100}%')

Epoch 1/100
34/34 [==============================] - 1s 12ms/step - loss: 0.7871 - accuracy: 0.7131 - val_loss: 0.8958 - val_accuracy: 0.7412
Epoch 2/100
34/34 [==============================] - 0s 3ms/step - loss: 0.7121 - accuracy: 0.7319 - val_loss: 0.8226 - val_accuracy: 0.7632
Epoch 3/100
34/34 [==============================] - 0s 4ms/step - loss: 0.6656 - accuracy: 0.7498 - val_loss: 0.7661 - val_accuracy: 0.7829
Epoch 4/100
34/34 [==============================] - 0s 4ms/step - loss: 0.6309 - accuracy: 0.7592 - val_loss: 0.7157 - val_accuracy: 0.7851
Epoch 5/100
34/34 [==============================] - 0s 5ms/step - loss: 0.6070 - accuracy: 0.7770 - val_loss: 0.6817 - val_accuracy: 0.8026
Epoch 6/100
34/34 [==============================] - 0s 4ms/step - loss: 0.5864 - accuracy: 0.7780 - val_loss: 0.6513 - val_accuracy: 0.8268
Epoch 7/100
34/34 [==============================] - 0s 5ms/step - loss: 0.5725 - accuracy: 0.7846 - val_loss: 0.6305 - val_accuracy: 0.8289
Epoch 8/100


## fungsi save model

In [5]:
model.save('my_subclassed_model', save_format="tf") 

INFO:tensorflow:Assets written to: my_subclassed_model\assets


## Memasukan data kedalam model yang sudah disimpan

### load sample data baru

In [6]:
loaded_model = tf.keras.models.load_model('my_subclassed_model') 

# Evaluate the loaded model
loss, accuracy = loaded_model.evaluate(X_test, y_test) 
print(f'Test accuracy: {accuracy * 100}%') 

15/15 [==============================] - 0s 1ms/step - loss: 0.5459 - accuracy: 0.8289
Test accuracy: 82.8947365283966%


### melakukan prediction menggunakan model yang sudah di load terhadap data baru dan hasil predictionnya dimasukan kedalam sample data baru

In [7]:
# predict data
# load new data
sampleNewData = pd.ExcelFile("sample new data.xlsx")
sampleNewData.sheet_names
newData = sampleNewData.parse("Sheet1")
newData.head()


,No,Nama Peserta,Lagu & Penyanyi Asli,Hasil,Genre,Babak,Musim,Usia,Gender,Asal Daerah
0,1,Helena Andrian,"Yang Ku 'Tunggu"" - Anggun",1,1,8,1,20,0,1


In [8]:
# Make predictions on new data 
scaler = MinMaxScaler(feature_range=(0, 1))
newData['Usia_Normalized'] = scaler.fit_transform(newData[['Usia']])
newData['Babak_Normalized'] = scaler.fit_transform(newData[['Babak']])

predictions = loaded_model.predict(newData[['Genre','Babak','Gender','Asal Daerah','Usia_Normalized','Babak_Normalized']].values) 
if predictions[0][1] > 0.5: 
    predicted_class = 1 
else:
    predicted_class = 0

newData['predictions'] = predicted_class
newData.head()

1/1 [==============================] - 0s 114ms/step


,No,Nama Peserta,Lagu & Penyanyi Asli,Hasil,Genre,Babak,Musim,Usia,Gender,Asal Daerah,Usia_Normalized,Babak_Normalized,predictions
0,1,Helena Andrian,"Yang Ku 'Tunggu"" - Anggun",1,1,8,1,20,0,1,0.0,0.0,1
